In [7]:
# from flask import Flask, request, jsonify
import openai
import json
import pg8000
from dotenv import load_dotenv
import os

# app = Flask(__name__)

# Load environment variables
load_dotenv(dotenv_path="./.env")

# OpenAI API Key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Database connection parameters
DATABASE_URL = os.getenv("POSTGRES_URL")
DATABASE_USER = os.getenv("POSTGRES_USER")
DATABASE_PASS = os.getenv("POSTGRES_PASSWORD")
DATABASE_DB = os.getenv("POSTGRES_DATABASE")
DATABASE_HOST = os.getenv("POSTGRES_HOST")


### Start with Connecting to DB, also functions for receiving from DB

In [8]:
def get_db_connection():
    """ Creates a Database connection """
    conn = pg8000.connect(
        user=DATABASE_USER,
        password=DATABASE_PASS,
        host=DATABASE_HOST,
        database=DATABASE_DB
    )
    return conn


def get_user_diet(uid, cursor):
    """
    Retrieves the User Diet Restrictions
    
    Inputs:\\
    uid - user Id\\
    cursor - database cursor object

    Returns:\\
    String of all assoicated dietary requirements
    """

    try:
        query = f"""
                SELECT dt."dietName" FROM userpreference as up
                JOIN diettype as dt ON up."dietId" = dt."dietId" 
                WHERE up.uid = {uid} 
                """
        cursor.execute(query)
        diets = cursor.fetchall()
        diet_list = [diet[0] for diet in diets]
        return ', '.join(diet_list)
    except Exception as e:
        print(f"Database error: {e}")
        return ""
    

def get_conversation_context(uid, cursor):
    """ 
    Gets context to give to chatbot 
    
    Inputs:\\
    uid - user Id\\
    cursor - database cursor object

    Returns:\\
    List of 10 most previous questions and answers formatted to give
    chatbot correct context
    """

    try:
        query = f"""
            SELECT "userQuestion", "chatResponse"
            FROM context
            WHERE uid = {uid}
            ORDER BY timestamp DESC
            LIMIT 10;
        """
        cursor.execute(query)
        context = cursor.fetchall()
        # Flatten the context into a list of messages
        messages = []
        for user_q, chat_resp in reversed(context):
            messages.append({"role": "user", "content": user_q})
            messages.append({"role": "assistant", "content": chat_resp})
        return messages
    except Exception as e:
        print(f"Database error: {e}")
        return []
    

def get_user_recipes(uid, cursor):
    """
    Retrieves the previous user recipes
    
    Inputs:\\
    uid - user Id\\
    cursor - database cursor object

    Returns:\\
    List of previous recipes names for context
    """

    try:
        query = f"""
            SELECT "recipeName"
            FROM recipe
            WHERE uid = {uid}; 
        """
        cursor.execute(query)
        recipes = cursor.fetchall()
        recipe_names = [recipe[0] for recipe in recipes]
        return recipe_names
    except Exception as e:
        print(f"Database error: {e}")
        return []


### Classify the message

In [9]:
def classify_message(user_question, user_diet, context, client):
    classification_rules = """
    You are to classify the user's request into one of the following categories:
    - "recipe": If the user is asking for a recipe.
    - "lifestyle": If the user is seeking lifestyle tips related to their diet.
    - "general": If the user has general questions about their dietary needs.
    - "not applicable": If the request is unrelated to diet or lifestyle.

    Respond with only one of the above categories.
    """

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", "content": classification_rules},
                {"role": "assistant", "content": f"Dietary preferences: {user_diet}, context: {context}"},
                {"role": "user", "content": user_question}
            ],
            max_tokens=5,
            n=1,
            stop=None,
            temperature=0
        )
        classification = completion.choices[0].message.content.strip().lower()
        return classification
    except Exception as e:
        print(f"Classification error: {e}")
        return None


### Generate a response

In [19]:
def generate_recipe(user_question, user_diet, context, previous_recipes, client):
    """
    Generates a new recipe for the user
    """
    response = client.chat.completions.create(
    model="gpt-4o-mini",  
    messages=[
        {
            "role": "system",
            "content": """You give recipes given certain dietary conditions. 
            You must make sure the recipe is not repeated.
            You must make sure the recipe does not recommend anything the user cannot eat
            with their dietary restrictions"""
        },
        {
            "role": "assistant",
            "content": f"""Dietary restrictions: User has the following diet requirements {user_diet},
                        Previous Recipes: {previous_recipes}"""
        },
        {
            "role": "user",
            "content": user_question
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recipe_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "Recipe": {
                        "description": "The name of the recipe",
                        "type": "string"
                    },
                    "Ingredients": {
                        "description": "list of ingredients with measurements if needed",
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                    "Prep Time": {
                        "description": "time in minutes to prepare this recipe",
                        "type": "integer"
                    },
                    "Instructions": {
                        "description": "Instructions to create this recipe, stored as JSON",
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                    "additionalProperties": False
            }
        }
    }}
    )

    # Print the response
    print(response.choices[0].message.content)
    recipe = json.loads(response.choices[0].message.content)
    return(recipe)

In [10]:
def generate_response(classification, user_question, user_diet, context, previous_recipes, client):
    if classification == "not applicable":
        return "I'm sorry, but I can only assist with dietary and lifestyle-related questions."

    # Base system prompt
    system_prompt = f"You are a helpful assistant specialized in {user_diet}."

    # Adjust the prompt based on classification
    if classification == "recipe":
        # Include the list of previous recipes to avoid repetition
        recipe_prompt = f"""
        Provide a new recipe that fits the following dietary requirements: {user_diet}.
        Do not provide any of the following recipes: {', '.join(previous_recipes)}.
        The recipe should be in JSON format with the following fields:
        - recipeName
        - ingredients (as a list)
        - prepTime (in minutes)
        - instructions
        """
        messages = [{"role": "system", "content": system_prompt},
                    *context,
                    {"role": "user", "content": recipe_prompt}]
    elif classification == "lifestyle":
        lifestyle_prompt = f"""
        Provide a lifestyle tip related to {user_diet}.
        """
        messages = [{"role": "system", "content": system_prompt},
                    *context,
                    {"role": "user", "content": lifestyle_prompt}]
    elif classification == "general":
        messages = [{"role": "system", "content": system_prompt},
                    *context,
                    {"role": "user", "content": user_question}]
    else:
        return "I'm sorry, I didn't understand your request."

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=messages,
            temperature=0.7
        )
        response = completion.choices[0].message.content.strip()
        return response
    except Exception as e:
        print(f"Response generation error: {e}")
        return "I'm sorry, I couldn't process your request at this time."


In [20]:
# Connect to Database
conn = get_db_connection()
cursor = conn.cursor()

# Connect to OpenAI
client = openai.OpenAI()

userId = 1



try:
    user_diet = get_user_diet(userId, cursor)
    context = get_conversation_context(userId, cursor)
    recipes = get_user_recipes(userId, cursor)

    user_message = "I want a recipe with chicken"

    # classify = classify_message(user_message, user_diet, context, client)
    # response = generate_response(classify, user_message, user_diet, context, recipes, client)
    recipe = generate_recipe(user_message, user_diet, context, recipes, client)
    print(recipe)
    # print(response)
    # print(user_diet)
    # print(context)
    # print(recipes)

except Exception as e:
    print("Exception:" + e)

finally:
    conn.close()

{"Recipe":"Herbed Grilled Chicken","Ingredients":["4 boneless, skinless chicken breasts","2 tablespoons olive oil","2 teaspoons dried oregano","2 teaspoons dried basil","1 teaspoon garlic powder","Salt to taste","Pepper to taste","Juice of 1 lemon"],"Prep Time":15,"Instructions":["In a bowl, mix olive oil, oregano, basil, garlic powder, salt, pepper, and lemon juice to create a marinade.","Place the chicken breasts in the marinade and coat evenly. Let it marinate for at least 30 minutes, or up to 2 hours for more flavor.","Preheat your grill or grill pan over medium-high heat.","Remove the chicken from the marinade and place it on the grill.","Grill the chicken for about 6-7 minutes on each side or until thoroughly cooked and the juices run clear.","Remove from the grill and let it rest for a few minutes before serving."]}
{'Recipe': 'Herbed Grilled Chicken', 'Ingredients': ['4 boneless, skinless chicken breasts', '2 tablespoons olive oil', '2 teaspoons dried oregano', '2 teaspoons dri

In [ ]:
r = json.loads(response)
r

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [89]:
conn = pg8000.connect(
    user=DATABASE_USER,
    password=DATABASE_PASS,
    host=DATABASE_HOST,
    database=DATABASE_DB
)
cursor = conn.cursor()
try:
    # Example query: Fetch User table
    cursor.execute('SELECT * FROM diettype')
    rows = cursor.fetchall()
    print(rows)

    # Example query: Fetch User table
    cursor.execute("""SELECT dt."dietName" FROM userpreference as up
                   JOIN diettype as dt ON up."dietId" = dt."dietId" 
                   WHERE up.uid = 1 """)
    rows = cursor.fetchall()
    print(rows)

    # # Fetch column names and their data types for each table
    # tables = ['users', 'UserPreference', 'DietType', 'Context', 'Tip', 'Recipe']

    # for table in tables:
    #     cursor.execute(f"""
    #         SELECT column_name, data_type 
    #         FROM information_schema.columns 
    #         WHERE table_name = '{table}';
    #     """)
    #     columns = cursor.fetchall()
    #     print(f"Columns and their types in '{table}' table:")
    #     for column_name, data_type in columns:
    #         print(f"  {column_name}: {data_type}")
    #     print()  # Blank line for readability
except Exception as e:
    print("Exception" + e)

    # Close the connection
finally:
    conn.close()

([1, 'GlutenFree'], [2, 'LowFodmap'], [3, 'Mediterranean'], [4, 'DASH'], [5, 'Anti-Inflammatory'], [6, 'LowCal-LowFat'])
(['GlutenFree'], ['Anti-Inflammatory'])
